### Notebook for trading_bot

Starting with paper trading

In [22]:
import sys
import os

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "../../"))
sys.path.insert(0, BASE_DIR)

import lumibot
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from datetime import datetime

import math

# News
from alpaca_trade_api import REST
from timedelta import Timedelta

# Strategies
from strategies.ML_models.finbert_utils import estimate_sentiment

In [23]:
API_KEY = "PK5VOHYTOFZB5OCY1CDE"
API_SECRET = "NxTEBhwXnCoKOeVXQ5TrWWQNRxiv3jU4UUyAZ8Y5"
BASE_URL = "https://paper-api.alpaca.markets/v2"

In [24]:
ALPACA_CREDS = {
    "API_KEY": API_KEY,
    "API_SECRET": API_SECRET,
    "PAPER": True
}

In [ ]:
class MLTrader(Strategy):
    def initialize(self, symbol:str="SPY", cash_at_risk:float=.5):
        self.symbol= symbol
        self.sleeptime = "24H" # dictates how frequently we're gonna trade
        self.last_trade = None
        self.cash_at_risk = cash_at_risk # Cash at risk: How much of our cash balance do we want to risk at every trade, 0.5 means for each trade we're using 50% of our remaining cash balance
        self.api = REST(base_url=BASE_URL, key_id=API_KEY, secret_key=API_SECRET)
    
    def position_sizing(self):
        cash = self.get_cash()
        last_price = self.get_last_price(self.symbol)
        
        # Calculating sizing
        quantity = math.floor(cash * self.cash_at_risk / last_price) # tells us how many units we're gonna get per amount that we want to risk
        return cash, last_price, quantity

    def get_dates(self):
        today = self.get_datetime()

        # Get 3 days worth of news
        three_days_prior = today - Timedelta(days=3)
        return today.strftime('%Y-%m-%d'), three_days_prior.strftime('%Y-%m-%d')

    def get_sentiment(self):
        today, three_days_prior = self.get_dates()
        news = self.api.get_news(symbol=self.symbol, 
                                 start=three_days_prior, 
                                 end=today)
        news = [ev.__dict__["_raw"]["headline"] for ev in news] # ev: event
        probability, sentiment = estimate_sentiment(news)

        return probability, sentiment

    def on_trading_iteration(self): # runs once everything we get a tick
        cash, last_price, quantity = self.position_sizing()
        probability, sentiment = self.get_sentiment()

        if cash > last_price: # check to make sure we're not buying when we don't have enough cash
            if sentiment == "positive" and probability > .999:
                if self.last_trade == "sell":
                    self.sell_all()
                order = self.create_order(
                    self.symbol,
                    quantity,
                    "buy",
                    type="bracket",

                    # stop loss
                    take_profit_price=last_price*1.2,
                    stop_loss_price=last_price*0.95
                )
                self.submit_order(order)
                self.last_trade = "buy"

            elif sentiment == "negative" and probability > .999:
                if self.last_trade == "buy":
                    self.sell_all()
                order = self.create_order(
                    self.symbol,
                    quantity,
                    "sell",
                    type="bracket",

                    # stop loss
                    take_profit_price=last_price*.8,
                    stop_loss_price=last_price*1.05
                )
                self.submit_order(order)
                self.last_trade = "sell"
            

In [ ]:
start_date = datetime(2020, 1, 1)
end_date = datetime(2023, 12, 31)

In [27]:
broker = Alpaca(ALPACA_CREDS)
strategy = MLTrader(name='mlstrat', broker=broker,
                    parameters={"symbol":"SPY",
                                "cash_at_risk":.5})

In [28]:
strategy.backtest(
    YahooDataBacktesting,
    start_date,
    end_date,
    parameters={"symbol":"SPY", "cash_at_risk":.5}
)

Progress || 100.00%  [Elapsed: 0:00:02 ETA: 0:00:00] Portfolio Val: 99,723.850

{'cagr': -0.06511865976446773,
 'volatility': 0.01035290336110209,
 'sharpe': -11.325195829808136,
 'max_drawdown': {'drawdown': 0.003171020507812483,
  'date': Timestamp('2023-12-28 09:30:00-0500', tz='America/New_York')},
 'romad': -20.535553019614373,
 'total_return': -0.0027615051269530566}